In [60]:
!pip install torch-summary

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms, models
from torchvision.models.resnet import BasicBlock, ResNet
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary

In [85]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class CustomModel(nn.Module):
    def __init__(self, block, num_blocks, num_classes=25):
        super(CustomModel, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

In [83]:
# take things to device the entire models things so thattheres no conflict between variables in devices

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)     


In [84]:
# Viem Model architecture
from torchsummary import summary
model = to_device(CustomModel(BasicBlock, [2, 2, 2, 2], num_classes=25),device)
summary(model)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─Sequential: 1-3                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─Conv2d: 3-3                  36,864
|    |    └─BatchNorm2d: 3-4             128
|    |    └─Sequential: 3-5              --
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─Conv2d: 3-8                  36,864
|    |    └─BatchNorm2d: 3-9             128
|    |    └─Sequential: 3-10             --
├─Sequential: 1-4                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            256
|    |    └─Conv2d: 3-13                 147,456
|    |    └─BatchNorm2d: 3-14        

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─Sequential: 1-3                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─Conv2d: 3-3                  36,864
|    |    └─BatchNorm2d: 3-4             128
|    |    └─Sequential: 3-5              --
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─Conv2d: 3-8                  36,864
|    |    └─BatchNorm2d: 3-9             128
|    |    └─Sequential: 3-10             --
├─Sequential: 1-4                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            256
|    |    └─Conv2d: 3-13                 147,456
|    |    └─BatchNorm2d: 3-14        

In [86]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4731, 0.4819, 0.4018], std=[0.1925, 0.1915, 0.1963])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4706, 0.4802, 0.4020], std=[0.1907, 0.1898, 0.1950])
    ]),
}

In [87]:
# Data loaded into the required part

data_dir = '/kaggle/input/seen-dataset-12-class/Seen Datasets'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x])
                  for x in ['train', 'val']}
data_loaders = {x: data.DataLoader(image_datasets[x], batch_size=batch_size,
                                   shuffle=True, num_workers=8, pin_memory=True)
                for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [88]:
# Loss and optimizer Intitalize

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=initial_lr,steps_per_epoch=len(data_loaders['train']), epochs=num_epochs)


In [89]:
# Hyperparameters and Device Initialization 
num_classes = 25
batch_size = 64
num_epochs = 50
grad_clip=0.1
initial_lr = 0.001
weight_decay = 1e-4
dropout_rate = 0.5
patience = 10 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Setup parameters required later for tracking 
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

# Training process not a fucntion
best_acc = 0.0
early_stop_counter = 0
best_model_wts = None
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(data_loaders[phase], desc=f"{phase} - Epoch {epoch+1}"):
            inputs = inputs.to(device, non_blocking=True) #Cuda or CPU
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
#               Removed  
#                 if grad_clip: 
#                     nn.utils.clip_grad_value_(model.parameters(), grad_clip)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()
            epoch_train_loss = running_loss / dataset_sizes['train']
            epoch_train_corrects = running_corrects.double() / dataset_sizes['train']
        else:
            epoch_val_loss = running_loss / dataset_sizes['val']
            epoch_val_corrects = running_corrects.double() / dataset_sizes['val']

    train_loss_history.append(epoch_train_loss)
    train_acc_history.append(epoch_train_corrects.item())
    val_loss_history.append(epoch_val_loss)
    val_acc_history.append(epoch_val_corrects.item())

    print(f'Train Loss: {epoch_train_loss:.4f} Accuracy: {epoch_train_corrects:.4f}')
    print(f'Validation Loss: {epoch_val_loss:.4f} Accuracy: {epoch_val_corrects:.4f}')

    if epoch_val_corrects > best_acc:
        best_acc = epoch_val_corrects
        best_model_wts = model.state_dict().copy()
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping done")
        model.load_state_dict(best_model_wts)
        break

print('Best Validation Accuracy: {:4f}'.format(best_acc))

# Load best model weights
if best_model_wts:
    model.load_state_dict(best_model_wts)
# Better if saved model each time we got better accuracy but takes too long time
# Save the entire model
torch.save(model, 'resnet18_image_classify.pth')
print("Model saved")

Epoch 0/49


val - Epoch 1: 100%|██████████| 118/118 [00:44<00:00,  2.64it/s]


Train Loss: 2.4209 Accuracy: 0.2863
Validation Loss: 2.1073 Accuracy: 0.3727
Epoch 1/49


val - Epoch 2: 100%|██████████| 118/118 [00:45<00:00,  2.61it/s]


Train Loss: 2.2422 Accuracy: 0.3363
Validation Loss: 1.9708 Accuracy: 0.4159
Epoch 2/49


...

val - Epoch 18: 100%|██████████| 118/118 [00:48<00:00,  2.44it/s]


Train Loss: 1.2849 Accuracy: 0.6203
Validation Loss: 0.8310 Accuracy: 0.7425
Epoch 18/49


val - Epoch 19: 100%|██████████| 118/118 [00:44<00:00,  2.63it/s]


Train Loss: 1.2480 Accuracy: 0.6285
Validation Loss: 0.7744 Accuracy: 0.7644
Epoch 19/49


train - Epoch 20:   5%|▍         | 16/352 [00:09<01:24,  3.96it/s]

In [ ]:
# Plot the training curves
epochs = len(train_loss_history)

plt.figure(figsize=(12, 5))

# Plot the loss
plt.subplot(1, 2, 1)
plt.plot(range(epochs), train_loss_history, label='Train Loss')
plt.plot(range(epochs), val_loss_history, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.title('Training and Validation Loss Graph')
plt.legend()

plt.show()

In [ ]:
# Plot the accuracy
plt.subplot(1, 2, 2)
plt.plot(range(epochs), train_acc_history, label='Train Accuracy')
plt.plot(range(epochs), val_acc_history, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Graph')
plt.legend()

plt.show()